The script inserts admission/discharge events to the corresponding positions and saves as output

In [ ]:
#!pip install modin[ray]

In [ ]:
#!export MODIN_OUT_OF_CORE=true

In [2]:
#import ray
#ray.init(num_cpus=8)
import modin.pandas as pd

In [3]:
#import pandas as pd
import re
import os
import json

In [4]:
FNAME = '20110101' #Enter either month/all
DATA_TYPE = 'original' #Enter either downsampled/original

SEQ_LEN = 1000

AD_DIS_PATH = "../../../data/AE_CDiff_d00845/output/data/1000/admission_discharge/results.json"

ALL_DATA_PATH = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed/{FNAME}.csv"
TRAIN_DATA_PATH = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed/splits/{FNAME}/train.csv"
VALID_DATA_PATH = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed/splits/{FNAME}/val.csv"
TEST_DATA_PATH = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed/splits/{FNAME}/test.csv"

OUT_ALL_DATA_PATH = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed_v2/{FNAME}.csv"
OUT_TRAIN_DATA_PATH = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed_v2/splits/{FNAME}/train.csv"
OUT_VALID_DATA_PATH = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed_v2/splits/{FNAME}/val.csv"
OUT_TEST_DATA_PATH = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed_v2/splits/{FNAME}/test.csv"

# OUT_VALID_DATA_PATH2 = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed_v2/splits/{FNAME}/val_imb.csv"
# OUT_TEST_DATA_PATH2 = f"../../../data/AE_CDiff_d00845/output/data/1000/{DATA_TYPE}/preprocessed_v2/splits/{FNAME}/test_imb.csv"

In [5]:
df = pd.read_csv(OUT_TEST_DATA_PATH, nrows=None)
df.shape

(248654, 1004)

In [6]:
df.head()

,999,998,997,996,995,994,993,992,991,990,...,5,4,3,2,1,0,d_00845,patient_id,num_gt_codes,has_gt_codes
0,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,d_5856_rf,h_82310,h_84100,1_days,d_5856_rf,h_90960,0,27NHU5AI9,50,1
1,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,h_Q5001,1_days,h_G0154,h_G0156,30_days,h_Q5001,0,WKDHL74CS,3,1
2,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,14_days,d_4619,h_99213,10_days,d_1534_rf,h_99213,0,Z9L8OZ723,2,1
3,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,2_days,discharge,21_days,d_1991,d_5119,h_71020,0,D0KJ7TJYX,4,1
4,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,4_days,d_72871,d_7397,d_84212,h_98927,h_99213,0,HQSH6B8KE,1,1


In [16]:
print(df.iloc[0].tolist())

['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

In [ ]:
#### Make Data Imbalanced (Val & Test)

In [ ]:
# df_val = pd.read_csv(OUT_VALID_DATA_PATH)
# df_test = pd.read_csv(OUT_TEST_DATA_PATH)

In [ ]:
# df_val.head()

In [ ]:
def imbalance(df, output_path, label='d_00845'):
    """Imbalance dataset 99/1 ratio."""
    class_0 = df[label].value_counts(normalize=False)[0]
    class_1 = int(class_0/99.0)
    df0 = df[df[label]==0]
    df1 = df[df[label]==1]
    df1 = df.sample(n=class_1)
    df_out = pd.concat([df0, df1], axis=0)
    #import pdb; pdb.set_trace()
    df_out = df_out.sort_values(by=['num_gt_codes'], ascending=False)
    #df_out = df_out.sample(frac=1)
    
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)
    
    df_out.to_csv(output_path, index=False)
    return df_out

In [ ]:
# df_val2 = imbalance(df_val, OUT_VALID_DATA_PATH2)

In [ ]:
# df_val2.head()

In [ ]:
# df_test2 = imbalance(df_test, OUT_TEST_DATA_PATH2)

In [ ]:
# df_test2.head(10)

In [ ]:
# df_test2[label].value_counts()

In [ ]:
#End...

In [ ]:
nrows = None
label = 'd_00845'

In [ ]:
df_train = pd.read_csv(TRAIN_DATA_PATH, nrows=None)

In [ ]:
df_val = pd.read_csv(VALID_DATA_PATH, nrows=None)

In [ ]:
df_test = pd.read_csv(TEST_DATA_PATH, nrows=None)

In [ ]:
df_train[label].value_counts(normalize=False), df_train[label].value_counts(normalize=True)

In [ ]:
df_val[label].value_counts(normalize=False), df_val[label].value_counts(normalize=True)

In [ ]:
df_test[label].value_counts(normalize=False), df_test[label].value_counts(normalize=True)

In [ ]:
# print(df_all.shape)
# df_all['d_00845'].value_counts(normalize=False)

In [ ]:
# df_train = pd.read_csv(TRAIN_DATA_PATH, nrows=None)
# df_val = pd.read_csv(VALID_DATA_PATH, nrows=None)
# df_test = pd.read_csv(TEST_DATA_PATH, nrows=None)

In [11]:
with open(AD_DIS_PATH, "r") as fp:
    dict_ad_dis = json.load(fp)

In [12]:
len(dict_ad_dis)

20050947

In [14]:
patient_id = df['patient_id'].iloc[0]
patient_id += '_' + '20110101'
patient_id

'27NHU5AI9_20110101'

In [15]:
dict_ad_dis[patient_id]

{'admission': [86], 'discharge': [79]}

In [ ]:

df_all.shape, df_train.shape, df_val.shape, df_test.shape

In [ ]:
df_all.value_counts(normalize=True), df_all.value_counts(normalize=False)

In [ ]:
def add_ad_dis(row0, dict_ad_dis, n_events=1000):
    """Add admission and discharge to the row if available."""
    patient_id = row0['patient_id']
    ad_dis = dict_ad_dis[patient_id]
    ad = ad_dis['admission']
    dis = ad_dis['discharge']
    cols = [str(i) for i in range(n_events-1, -1, -1)]
    all_cols = list(row0.index)
    if ad or dis:
        row = row0[cols].tolist()
        row.reverse()
        for indx in ad:
            row.insert(indx, 'admission')
        for indx in dis:
            row.insert(indx, 'discharge')
        row = row[:n_events]
        row.reverse()
        row0[cols] = row[:]
    return row0    

In [ ]:
df_all2 = df_all.apply(add_ad_dis, args=(dict_ad_dis, SEQ_LEN), axis=1)
df_train2 = df_train.apply(add_ad_dis, args=(dict_ad_dis, SEQ_LEN), axis=1)
df_val2 = df_val.apply(add_ad_dis, args=(dict_ad_dis, SEQ_LEN), axis=1)
df_test2 = df_test.apply(add_ad_dis, args=(dict_ad_dis, SEQ_LEN), axis=1)

In [ ]:
df_all2.shape, df_train2.shape, df_val2.shape, df_test2.shape

In [ ]:
output_dir1 = os.path.dirname(OUT_ALL_DATA_PATH)
output_dir2 = os.path.dirname(OUT_TRAIN_DATA_PATH)

os.makedirs(output_dir1, exist_ok=True)
os.makedirs(output_dir2, exist_ok=True)

In [ ]:
df_all2.to_csv(OUT_ALL_DATA_PATH, index=False)
df_train2.to_csv(OUT_TRAIN_DATA_PATH, index=False)
df_val2.to_csv(OUT_VALID_DATA_PATH, index=False)
df_test2.to_csv(OUT_TEST_DATA_PATH, index=False)